In [7]:
import warnings 
warnings.filterwarnings("ignore")
import pandas as pd
import numpy as np 
import seaborn as sns
import matplotlib.pyplot as plt
from wordcloud import WordCloud
import re
import pickle
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
stop = set(stopwords.words('english'))
from nltk.tokenize import word_tokenize
from textblob import TextBlob
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer("english")
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import confusion_matrix,classification_report,accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

[nltk_data] Error loading stopwords: <urlopen error [WinError 10060] A
[nltk_data]     connection attempt failed because the connected party
[nltk_data]     did not properly respond after a period of time, or
[nltk_data]     established connection failed because connected host
[nltk_data]     has failed to respond>


ImportError: cannot import name '__version__' from 'sklearn' (unknown location)

In [3]:
!pip install pandas
!pip install seaborn
#import nltk
#nltk.download()

     ---------------------------------------- 0.0/293.3 kB ? eta -:--:--
     - -------------------------------------- 10.2/293.3 kB ? eta -:--:--
     - -------------------------------------- 10.2/293.3 kB ? eta -:--:--
     - -------------------------------------- 10.2/293.3 kB ? eta -:--:--
     - -------------------------------------- 10.2/293.3 kB ? eta -:--:--
     --------- --------------------------- 71.7/293.3 kB 328.6 kB/s eta 0:00:01
     ----------- ------------------------- 92.2/293.3 kB 327.7 kB/s eta 0:00:01
     ------------- ---------------------- 112.6/293.3 kB 364.4 kB/s eta 0:00:01
     --------------- -------------------- 122.9/293.3 kB 343.4 kB/s eta 0:00:01
     --------------------- -------------- 174.1/293.3 kB 420.1 kB/s eta 0:00:01
     ------------------------- ---------- 204.8/293.3 kB 461.0 kB/s eta 0:00:01
     ---------------------------- ------- 235.5/293.3 kB 465.5 kB/s eta 0:00:01
     --------------------------------- -- 276.5/293.3 kB 501.4 kB/s eta

In [ ]:
df = pd.read_csv('tripadvisor_hotel_reviews.csv')
df.head()

In [ ]:
df.shape

In [ ]:
df.info()

In [ ]:
df.describe(include='all')

In [ ]:
df.isnull().sum()

In [ ]:
sns.boxplot(x = df['Rating'])

In [ ]:
sns.set_theme(style="white", color_codes=True)
sns.countplot(x = df['Rating'], palette=['pink', 'Aqua','coral','teal', 'lime']) 

In [ ]:
df['Word_count']= df['Review'].map(lambda x: len(x.split()))

In [ ]:
df.head(1)

In [ ]:
sns.boxplot(x = 'Rating', y='Word_count', data=df)

In [ ]:
sns.lineplot(x='Rating', y='Word_count', data=df)

In [ ]:
#pip install textblob

In [ ]:
#pip install wordcloud

In [ ]:
def polarity(x):
    blob = TextBlob(x)
    return blob.sentiment.polarity

In [ ]:
def subjectivity(x):
    blob = TextBlob(x)
    return blob.sentiment.subjectivity

In [ ]:
df['polarity']= df['Review'].apply(polarity)
df['Subjectivity']= df['Review'].apply(subjectivity)

In [ ]:
df.head(1)

In [ ]:
conditions = [
    (df['polarity']>0),
    (df['polarity']<0),
    (df['polarity']==0)
]
values = ['Positive','Negative','Nuetral']
df['polarity_new'] = np.select(conditions,values) #distribute 

In [ ]:
sns.distplot(df['polarity'])

In [ ]:
sns.distplot(df['Subjectivity'])

In [ ]:
sns.countplot(x='polarity_new',hue='Rating',data=df, palette="Set1")

In [ ]:
sns.boxplot(x='Rating',y='polarity', data=df, whis=2.5, fliersize= 5)

In [ ]:
sns.boxplot(x='Rating',y='Subjectivity', data=df, whis=2)

In [ ]:
df['Review'] = df['Review'].apply(lambda x:' '.join([word for word in x.split() if word not in (stop)]))

In [ ]:
df.head()

In [ ]:
df['Review_stemming'] = df['Review'].str.split()
pd.set_option('display.max_colwidth', -1)
df['Review_stemming'] = df['Review_stemming'].apply(lambda x: [stemmer.stem(y) for y in x])

In [ ]:
df.head(1)

In [ ]:
w_tokenizer = nltk.tokenize.WhitespaceTokenizer()
lemmatizer = nltk.stem.WordNetLemmatizer()

def lemmatize_text(text):
    return [lemmatizer.lemmatize(w) for w in w_tokenizer.tokenize(text)]
df['Review_lemmatize'] = df.Review.apply(lemmatize_text)

In [ ]:
df.head(1)

In [ ]:
def make_sentences(data,name):
    data[name]=data[name].apply(lambda x:' '.join([i+' ' for i in x]))
    data[name]=data[name].apply(lambda x:re.sub(r'\s+', ' ', x, flags=re.I))

In [ ]:
make_sentences(df,'Review_stemming')
make_sentences(df,'Review_lemmatize')

In [ ]:
df.head(1)

In [ ]:
rev_stemming = " ".join([review for review in df['Review_stemming']])
rev_stemming[:2000]

In [ ]:
rev_lmtz = " ".join([review for review in df['Review_lemmatize']])
rev_lmtz[:2000]

In [ ]:
plt.figure(figsize=(12,7))
wc= WordCloud(max_words=200,height= 800, width=1000 ,background_color='black').generate(rev_stemming)
plt.imshow(wc)

In [ ]:
plt.figure(figsize=(12,7))
wc= WordCloud(max_words=200,height= 800, width=1000 ,background_color='black').generate(rev_lmtz)
plt.imshow(wc)

In [ ]:
def sentiment(review):
    if review>=3:
        return 1
    else:
        return 0
df['Sentiment']= df['Rating'].apply(sentiment)

In [ ]:
df.head(1)

In [ ]:
X= df['Review_stemming']
Y= df['Sentiment']

X_train, X_test,y_train, y_test= train_test_split(X, Y, test_size=0.25, stratify=Y)

In [ ]:
tfidf= TfidfVectorizer(max_features=10000, tokenizer= word_tokenize,ngram_range=(1,2) )
X_train_tfidf= tfidf.fit_transform(X_train.values)
X_test_tfidf= tfidf.transform(X_test.values)

In [ ]:
pickle.dump(tfidf, open('tfidf.pkl', 'wb'))

In [ ]:
lr= LogisticRegression()
lr.fit(X_train_tfidf, y_train)
y_pred_tfidf = lr.predict(X_test_tfidf)

In [ ]:
pickle.dump(lr, open('trip_review.pkl', 'wb'))

In [ ]:
print(confusion_matrix(y_test,y_pred_tfidf))
print(accuracy_score(y_test,y_pred_tfidf))
print(classification_report(y_test,y_pred_tfidf))

In [ ]:
clf = RandomForestClassifier(n_estimators=20, random_state=0)
clf.fit(X_train_tfidf,y_train)
y_pred = clf.predict(X_test_tfidf)

print(confusion_matrix(y_test,y_pred))  
print(classification_report(y_test,y_pred))  
print(accuracy_score(y_test, y_pred))

In [ ]:
X_train_tfidf = X_train_tfidf.todense()
X_test_tfidf = X_test_tfidf.todense()

In [ ]:
model = pickle.load(open('trip_review.pkl','rb'))
tfidf = pickle.load(open('tfidf.pkl','rb'))

In [ ]:
def review_predictor(text):
    df =pd.DataFrame([[text]],columns=['Review'])
    df['Review'] = df['Review'].apply(lambda x:' '.join([word for word in x.split() if word not in (stop)]))
    df['Review'] = df['Review'].str.split()
    df['Review'] = df['Review'].apply(lambda x: [stemmer.stem(y) for y in x])
    df['Review'] = df['Review'].apply(lambda x:' '.join([i+' ' for i in x]))
    df['Review'] = df['Review'].apply(lambda x:re.sub(r'\s+', ' ', x, flags=re.I))
    X = df['Review']
    X = tfidf.transform(X)
    X = model.predict(X)
    X = X[0]
    return X

In [ ]:
test = input("Please enter a Review:\n")
predict = review_predictor(test)
if predict == 0:
    print("Negative Review")
else:
    print("Positive Review")